In [1]:
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import numpy as np
import cv2
import pyautogui
import pims
import trackpy as tp
from utils.tp_plots import *  # module of modified trackpy plotting functions
from utils.vector_utils import *
%matplotlib tk

In [ ]:
screenshot = pyautogui.screenshot()

fig, ax = plt.subplots()
ax.imshow(screenshot)

LEFT, TOP, WIDTH, HEIGHT = 0, 0, 0, 0

def line_select_callback(eclick, erelease):
    global LEFT, TOP, WIDTH, HEIGHT
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata

    LEFT = min(x1, x2)
    TOP = min(y1, y2)
    WIDTH = np.abs(x1 - x2)
    HEIGHT = np.abs(y1 - y2)

    rect = plt.Rectangle(
        (min(x1, x2), min(y1, y2)), np.abs(x1 - x2), np.abs(y1 - y2), fill=False
    )


rs = RectangleSelector(
    ax,
    line_select_callback,
    drawtype="box",
    useblit=False,
    button=[1],
    minspanx=5,
    minspany=5,
    spancoords="pixels",
    interactive=True,
)

plt.show()


In [3]:
while True:
    img = pyautogui.screenshot(region = (LEFT, TOP, WIDTH, HEIGHT))
    frame = np.array(img)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    pim_gray = pims.frame.Frame(gray)   # turn grayscale into PIMS Frame object

    f = tp.locate(pim_gray, 13, minmass=1000, invert=True)    # get feature location DataFrame
    fig = annotate(f, pim_gray).figure  # Get plt figure generated by trackpy's annotate

    # remove margins and axes from figure to only get annotated figure image
    ax = fig.gca()
    ax.axis('off')
    fig.tight_layout(pad=0)
    ax.margins(0)

    # Draw image onto plt canvas and convert into a numpy ndarray
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    cv2.imshow('image', image_from_plot) # show annotated image
    fig.clear(True) # clear canvas for next preprocessed image

KeyboardInterrupt: 